# Investigation into Sklearn Pipelines for Scaling and Model Selection

Using http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

and data https://archive.ics.uci.edu/ml/datasets/APS+Failure+at+Scania+Trucks

In [16]:
import numpy as np
import pandas as pd
import re
import os
from pandas.plotting import scatter_matrix

get_ipython().magic(u'env OMP_NUM_THREADS=2')

from IPython.display import display, HTML

import sklearn
import sklearn.model_selection
import requests
import io
import random

# Set the ransom seed used for the whole program to allow reprocibility
np.random.seed(3214412)

DEBUG = True # If true, pull a sample of the dataset for development 

env: OMP_NUM_THREADS=2


In [7]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00421/aps_failure_training_set.csv"
contents=requests.get(data_url).content

In [19]:
# First 20 rows of the file is a header with licensing info
# The header with column names is on row 21
raw_data_df=pd.read_csv(io.StringIO(contents.decode('utf-8')), skiprows=20, na_values="na")
raw_data_df.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,neg,33058,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,neg,41040,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,neg,12,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,neg,60874,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [27]:
# This is a poc on sklearn-pipelines so drop down to 10 rows
# Grab the 10 columns with the least number of null values and column "class"
data_df = raw_data_df[raw_data_df.isnull().sum().sort_values()[:11].index].dropna()
data_df.head()

,class,aa_000,bt_000,ck_000,cj_000,ci_000,by_000,aq_000,bi_000,bj_000,ao_000
0,neg,76698,76698.08,916567.68,0.0,5245752.00,43566.0,1132040.0,947550.0,799478.0,3655166.0
1,neg,33058,33057.51,643536.96,0.0,2291079.36,17733.0,338544.0,688314.0,392208.0,2127150.0
2,neg,41040,41040.08,236099.52,0.0,2322692.16,15439.0,153698.0,160176.0,139730.0,2173634.0
3,neg,12,12.69,4525.44,0.0,2135.04,32.0,1014.0,7632.0,3090.0,7554.0
4,neg,60874,60874.03,379111.68,0.0,3565684.80,24793.0,551022.0,653692.0,399410.0,2618878.0


In [35]:
print("Count of rows: {}".format(data_df.shape[0]))
print("Count of rows with class 'neg': {}".format(data_df[data_df['class'] == 'neg']['class'].shape[0]))
print("Count of rows with class 'pos': {}".format(data_df[data_df['class'] == 'pos']['class'].shape[0]))

Count of rows: 58888
Count of rows with class 'neg': 57932
Count of rows with class 'pos': 956


In [43]:
# Describe all the columns at once
pd.concat([data_df[col].describe().to_frame(name=col) for col in data_df.columns if col != 'class'], axis=1)

,aa_000,bt_000,ck_000,cj_000,ci_000,by_000,aq_000,bi_000,bj_000,ao_000
count,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04,5.888800e+04
mean,5.942603e+04,5.942603e+04,7.128344e+05,9.993976e+04,3.484558e+06,2.201318e+04,4.441264e+05,4.943652e+05,5.123684e+05,3.012701e+06
std,1.452684e+05,1.452684e+05,2.164271e+06,1.090789e+06,8.345773e+06,5.382452e+04,1.264976e+06,1.489898e+06,1.826194e+06,6.830351e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.760000e+02,8.751775e+02,1.470456e+04,0.000000e+00,4.921608e+04,2.190000e+02,4.167500e+03,1.596950e+04,8.519500e+03,6.570800e+04
50%,3.095200e+04,3.095214e+04,2.524493e+05,0.000000e+00,1.869525e+06,1.271800e+04,1.800570e+05,1.806820e+05,1.551150e+05,1.649212e+06
75%,4.889850e+04,4.889776e+04,5.514674e+05,0.000000e+00,2.956215e+06,2.041100e+04,3.785925e+05,3.812195e+05,3.349440e+05,2.686392e+06
max,2.746564e+06,2.746565e+06,5.542867e+07,6.094967e+07,1.409861e+08,1.002003e+06,2.556265e+07,4.493750e+07,4.573632e+07,1.222018e+08
